In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=x)

In [4]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten_1 True
20 dense_1 True
21 batch_normalization_1 True
22 dropout_1 True
23 dense_2 True
24 batch_normalization_2 True
25 dropout_2 True
26 dense_3 True


In [3]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.15,
                                         rotation_range=15,
                                         height_shift_range=0.15,
                                         width_shift_range=0.15,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [6]:
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    #optimizer=optimizers.Adam(lr=1e-3),
                    optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_fc.h5')

Epoch 1/10
156/156 [==============================] - 344s 2s/step - loss: 1.4764 - acc: 0.5324 - val_loss: 0.8617 - val_acc: 0.7374
Epoch 2/10
156/156 [==============================] - 345s 2s/step - loss: 0.5588 - acc: 0.8134 - val_loss: 0.9016 - val_acc: 0.7326
Epoch 3/10
156/156 [==============================] - 337s 2s/step - loss: 0.4024 - acc: 0.8652 - val_loss: 1.1008 - val_acc: 0.6895
Epoch 4/10
156/156 [==============================] - 337s 2s/step - loss: 0.3150 - acc: 0.8947 - val_loss: 1.2899 - val_acc: 0.6381


In [4]:
vgg16_model = load_model('vgg16_model_fc.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune')])
#                                    EarlyStopping(patience=10)])

Epoch 1/50
156/156 [==============================] - 531s 3s/step - loss: 1.8093 - acc: 0.4558 - val_loss: 3.2734 - val_acc: 0.3216
Epoch 2/50
156/156 [==============================] - 363s 2s/step - loss: 0.5128 - acc: 0.8310 - val_loss: 4.1825 - val_acc: 0.3057
Epoch 3/50
156/156 [==============================] - 364s 2s/step - loss: 0.2550 - acc: 0.9215 - val_loss: 0.7947 - val_acc: 0.7513
Epoch 4/50
156/156 [==============================] - 356s 2s/step - loss: 0.1754 - acc: 0.9458 - val_loss: 0.6117 - val_acc: 0.8227
Epoch 5/50
156/156 [==============================] - 360s 2s/step - loss: 0.1401 - acc: 0.9568 - val_loss: 1.6174 - val_acc: 0.6764
Epoch 6/50
156/156 [==============================] - 357s 2s/step - loss: 0.0968 - acc: 0.9724 - val_loss: 1.0122 - val_acc: 0.7657
Epoch 7/50
156/156 [==============================] - 355s 2s/step - loss: 0.0769 - acc: 0.9765 - val_loss: 1.7023 - val_acc: 0.6503
Epoch 8/50
156/156 [==============================] - 356s 2s/step - 

In [5]:
vgg16_model.save('vgg16_adam_50epoch')

In [6]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=6)])

Epoch 1/50
156/156 [==============================] - 365s 2s/step - loss: 0.0075 - acc: 0.9975 - val_loss: 0.7186 - val_acc: 0.8776
Epoch 2/50
156/156 [==============================] - 354s 2s/step - loss: 0.0090 - acc: 0.9969 - val_loss: 0.6612 - val_acc: 0.8837
Epoch 3/50
156/156 [==============================] - 356s 2s/step - loss: 0.0079 - acc: 0.9977 - val_loss: 0.6605 - val_acc: 0.8820
Epoch 4/50
156/156 [==============================] - 354s 2s/step - loss: 0.0077 - acc: 0.9972 - val_loss: 0.6876 - val_acc: 0.8802
Epoch 5/50
156/156 [==============================] - 353s 2s/step - loss: 0.0064 - acc: 0.9979 - val_loss: 0.7605 - val_acc: 0.8659
Epoch 6/50
156/156 [==============================] - 353s 2s/step - loss: 0.0067 - acc: 0.9978 - val_loss: 0.7678 - val_acc: 0.8641
Epoch 7/50
156/156 [==============================] - 354s 2s/step - loss: 0.0068 - acc: 0.9975 - val_loss: 0.7761 - val_acc: 0.8619
Epoch 8/50
156/156 [==============================] - 349s 2s/step - 

In [7]:
vgg16_model.save('vgg16_final_model.h5')

In [12]:
# vgg16_model.optimizer.lr = 1e-4

# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=5,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size)

In [13]:
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten_2 True
20 dense_4 True
21 batch_normalization_3 True
22 dropout_3 True
23 dense_5 True
24 batch_normalization_4 True
25 dropout_4 True
26 dense_6 True


In [14]:
    
# vgg16_model.compile(loss='categorical_crossentropy',
# #                     optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),#,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(lr=1e-3),
#                     metrics=['accuracy'])  

# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=20,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
#                                    CSVLogger('vgg16_fine_tune'),
#                                    EarlyStopping(patience=5)])


# vgg16_model.save('vgg16_fine_tuning_model.h5')

In [15]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

vgg16_model.save('vgg16_fine_tuning_model_SDG.h5')

Epoch 1/50
156/156 [==============================] - 348s 2s/step - loss: 0.1455 - acc: 0.9553 - val_loss: 0.8975 - val_acc: 0.7374
Epoch 2/50
156/156 [==============================] - 344s 2s/step - loss: 0.1055 - acc: 0.9665 - val_loss: 0.9091 - val_acc: 0.7339
Epoch 3/50
156/156 [==============================] - 343s 2s/step - loss: 0.0936 - acc: 0.9726 - val_loss: 0.8064 - val_acc: 0.7517
Epoch 4/50
156/156 [==============================] - 344s 2s/step - loss: 0.0832 - acc: 0.9743 - val_loss: 0.9933 - val_acc: 0.7139
Epoch 5/50
156/156 [==============================] - 344s 2s/step - loss: 0.0790 - acc: 0.9768 - val_loss: 0.9058 - val_acc: 0.7361
Epoch 6/50
156/156 [==============================] - 342s 2s/step - loss: 0.0682 - acc: 0.9796 - val_loss: 0.8920 - val_acc: 0.7417
Epoch 7/50
156/156 [==============================] - 342s 2s/step - loss: 0.0650 - acc: 0.9803 - val_loss: 0.9423 - val_acc: 0.7217
Epoch 8/50
156/156 [==============================] - 342s 2s/step - 

In [18]:
for layer in vgg16_model.layers:
    layer.trainable = True
    
for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_2 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_pool True
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten_2 True
20 dense_4 True
21 batch_normalization_3 True
22 dropout_3 True
23 dense_5 True
24 batch_normalization_4 True
25 dropout_4 True
26 dense_6 True


In [19]:
# vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
# #                     optimizer=optimizers.Adam(lr=1e-3),
#                     metrics=['accuracy'])  

# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=20,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
#                                    CSVLogger('vgg16_fine_tune'),
#                                    EarlyStopping(patience=5)])

# vgg16_model.save('vgg16_fine_tuning_model_all.h5')

Epoch 1/20
  8/156 [>.............................] - ETA: 15:02 - loss: 0.0502 - acc: 0.9863

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_3/SGD/gradients/block1_pool_1/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@block1_pool_1/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv2_1/Relu, block1_pool_1/MaxPool, training_3/SGD/gradients/block2_conv1_1/convolution_grad/Conv2DBackpropInput)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_3/SGD/gradients/block1_pool_1/MaxPool_grad/MaxPoolGrad', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/p